# December 22, 2023

https://adventofcode.com/2023/day/22

In [38]:
from collections import defaultdict

In [83]:
text = f'''1,0,1~1,2,1
0,0,2~2,0,2
0,2,3~2,2,3
0,0,4~0,2,4
2,0,5~2,2,5
0,1,6~2,1,6
1,1,8~1,1,9'''

test_text = text.split("\n")

In [84]:
fn = "data/22.txt"
with open(fn, "r") as file:
    text = file.readlines()

puzz_text = [x.strip() for x in text]

In [85]:
class Block:
    #block_count = 0

    def __init__(self, spec, id):
        # parse the coords
        xyz = spec.split("~")
        x0 = [int(x) for x in xyz[0].split(",")]
        x1 = [int(x) for x in xyz[1].split(",")] 

        self.x = [x0[0], x1[0]]
        self.y = [x0[1], x1[1]]
        self.z = [x0[2], x1[2]]
        self.id = id

        # remember which direction the block extends in
        # defaults to z for a 1x1
        if self.x[1] == self.x[0]:
            self.direc = "x"
        elif self.y[1] == self.y[0]:
            self.direc = "y"
        else:
            # self.z[1] == self.z[0]:
            self.direc = "z"

        # normalize the coords
        if (self.direc == "x" and (self.x[1] < self.x[0])):
            self.x = self.x[::-1]
        elif (self.direc == "y" and (self.y[1] < self.y[0])):
            self.y = self.y[::-1]
        elif (self.z[1] < self.z[0]):
            self.z = self.z[::-1]

    def __lt__(self, other):
        return self.z[0] < other.z[0]
    
    def __eq__(self, other):
        return self.z[0] == other.z[0]
    
    def __hash__(self):
        return hash(self.id)

In [211]:
class Puzzle:
    def __init__(self, text, xwid=10, ywid=10):
        # list of blocks in order from lowest to highest
        self.block_list = [Block(line, i) for i, line in enumerate(text)]
        self.block_list.sort()
        # to simplify part 2, make sure ids match position in list
        for i, b in enumerate(self.block_list):
            b.id = i

        # track highest level of for each [y][x] coord
        self.top = []
        for i in range(ywid):
            self.top.append( [0]*xwid )

        # track which block is in each [y][x], keyed by height z
        # this only tracks dropped blocks
        self.levels = defaultdict(lambda : [ [None]*xwid for i in range(ywid) ] )
        self.drop_all_blocks()

        '''Part 2'''
        # above[id] gives the set of blocks sitting on that block
        # below[id] gives the set of blocks that block is sitting on
        self.above = self.below = None


    def drop_all_blocks(self):
        '''drop all blocks from lowest to highest'''
        for b in self.block_list:
            self.dropblock(b)

    def dropblock(self, b):
        '''drop a block until it hits another block'''

        # find landing level
        max_height = 0
        for x in range(b.x[0], b.x[1]+1):
            for y in range(b.y[0], b.y[1]+1):
                max_height = max( max_height, self.top[y][x] )

        # adjust block's coords
        fall = b.z[0] - (max_height+1)
        b.z = [b.z[0]-fall, b.z[1]-fall]

        # adjust topography
        for x in range(b.x[0], b.x[1]+1):
            for y in range(b.y[0], b.y[1]+1):
                self.top[y][x] = max(self.top[y][x], b.z[1])

        # adjust self.levels
        for z in range(b.z[0], b.z[1]+1):
            for x in range(b.x[0], b.x[1]+1):
                for y in range(b.y[0], b.y[1]+1):
                    self.levels[z][y][x] = b.id

    def find_unsafe_blocks(self):
        '''find which blocks cannot safely be disintegrated'''
        unsafe = set()
        for b in self.block_list:
            supports = self.look_out_below(b)
            if len(supports) == 1:
                unsafe.add(supports.pop() )
        
        return unsafe
    
    def look_out_below(self, b):
        if b.z[0] == 1:
            return set()
        
        supports = set()
        for x in range(b.x[0], b.x[1]+1):
            for y in range(b.y[0], b.y[1]+1):
                below = self.levels[b.z[0]-1][y][x]
                if below is not None:
                    supports.add(below)

        return supports
    
    # For Part 2
    def create_support_graphs(self):
        self.above = {}
        self.below = {}
        for b in self.block_list:
            self.above[ b.id ] = self.look_up( b )
            self.below[ b.id ] = self.look_out_below( b )
        

    def look_up(self, b):
        above = set()
        for x in range(b.x[0], b.x[1]+1):
            for y in range(b.y[0], b.y[1]+1):
                up = self.levels[b.z[1]+1][y][x]
                if up is not None:
                    above.add(up)

        return above
    
    def melt_block(self, b):
        # blocks that will fall, so we need to check above them
        # temporarily seed it with current block
        # we remove it later since technically it is destroyed instead of falling
        falling = set([b.id])

        # blocks we've already checked
        fell = set()

        while len(falling) > 0:
            # feel like I should probably switch this to a set
            id = falling.pop()
            fell.add(id)
            
            up = self.above[id]
            for up_id in up:
                # I think this isn't necessary since the set should be sorted, but no harm in keeping it
                if up_id not in fell:
                    below_that = self.below[up_id]
                    if below_that <= falling.union(fell):
                        # see if we already checked this block to avoid recursion
                        falling.add(up_id)

        fell.remove(b.id)

        return len(fell), fell



                
                




### Part 1

In [212]:
test = Puzzle(test_text)

In [213]:
test.find_unsafe_blocks()

{0, 5}

In [214]:
len(test.block_list) - len(test.find_unsafe_blocks())

5

In [215]:
puzz = Puzzle(puzz_text)

In [216]:
len(puzz.block_list) - len(puzz.find_unsafe_blocks())

421

### Part 2

In [217]:
test = Puzzle(test_text)
test.create_support_graphs()

In [218]:
test.above

{0: {1, 2}, 1: {3, 4}, 2: {3, 4}, 3: {5}, 4: {5}, 5: {6}, 6: set()}

In [219]:
test.below

{0: set(), 1: {0}, 2: {0}, 3: {1, 2}, 4: {1, 2}, 5: {3, 4}, 6: {5}}

In [220]:
test.melt_block(test.block_list[0])

(6, {1, 2, 3, 4, 5, 6})

In [221]:
unsafe = test.find_unsafe_blocks()
unsafe

{0, 5}

In [222]:
tot = 0
for id in unsafe:
    num, blocks = test.melt_block( test.block_list[id] )
    tot += num
tot

7

In [223]:
puzz = Puzzle(puzz_text)
puzz.create_support_graphs()
unsafe = puzz.find_unsafe_blocks()
tot = 0
for id in unsafe:
    chain, dropped = puzz.melt_block( puzz.block_list[id] )
    print(id, chain)
    tot += chain
tot

1 86
2 52
3 2
5 1
6 3
8 6
12 32
14 51
15 2
16 1
17 25
19 85
20 1
22 1
23 50
24 5
25 24
26 1
28 2
29 3
30 3
31 1
35 23
36 84
37 45
39 22
40 2
42 1
43 4
45 71
46 2
47 8
51 21
52 43
53 1
54 20
55 4
57 65
60 2
62 19
63 2
64 2
65 1
66 42
67 6
69 4
71 64
72 1
74 40
75 10
78 3
79 1
81 4
82 3
83 9
84 7
85 51
86 3
87 3
88 22
90 2
91 8
93 2
94 1
95 1
97 6
102 1
105 5
106 3
107 14
108 1
110 13
111 2
112 3
113 6
114 3
115 2
116 4
118 3
120 1
121 50
122 3
125 2
126 2
127 1
128 2
129 5
130 1
131 167
132 1
134 4
135 1
136 49
138 1
139 2
140 1
142 12
143 164
145 162
146 2
147 48
149 161
150 1
152 3
155 9
156 11
157 2
158 1
159 47
160 5
161 46
162 160
163 2
164 6
165 2
166 34
169 1
171 1
172 123
173 5
174 1
175 4
176 32
179 1
180 43
181 1
183 107
185 14
189 2
190 31
191 1
192 3
194 13
195 3
196 5
197 20
198 19
201 2
203 42
204 4
205 11
206 12
208 106
209 2
210 25
212 1
213 99
214 10
215 18
216 1
217 5
219 16
222 10
223 24
224 1
225 93
226 7
227 4
228 1
230 2
232 22
233 2
236 9
237 9
238 1
239 5
241 1
2

39247

In [190]:
0 in unsafe

False

In [189]:
puzz.above[0]

set()

In [ ]:
# Oops
# too high: 42967 
# Error: I assume the melted block was unsafe (okay), but that doesn't mean ALL blocks above it will fall!

In [206]:
num, blocks = puzz.melt_block(puzz.block_list[1242])

In [207]:
num, blocks

(6, {1246, 1251, 1252, 1253, 1254, 1256})

In [200]:
puzz.above[1242]#, puzz.above[1254], puzz.above[1256]

{1246, 1251}

In [201]:
puzz.below[1246], puzz.below[1251]

({1242}, {1242, 1243})

In [202]:
puzz.above[1246]#, puzz.above[45]

{1252}

In [208]:
puzz.above[1252], puzz.below[1253]

(set(), {1252})